# Natural Language Processing + Decision Science

In this exercise, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import text_unidecode as unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?


## (0) Setup 🔨

First we'll load up the DataFrame which contains all the relevant information on Olist reviews!

In [3]:
df = pd.read_csv("olist_reviews.csv")

In [4]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review
0,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela ...
2,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
3,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews. 

**Beware NLTK has no Portuguese lemmatizer** (but it does have a stemmer `nltk.stem.RSLPStemmer`. 

In [5]:
from nltk.stem import RSLPStemmer
nltk.download("rslp")
stemmer = RSLPStemmer()

[nltk_data] Downloading package rslp to /home/aygul_unix/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('portuguese')) ## defining stopwords

def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.lower() ## lowercasing
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    # Advanced cleaning
    tokenized_sentence = word_tokenize(sentence) ## tokenizing

    tokenized_sentence = [w for w in tokenized_sentence
                                  if not w in stop_words] ## remove stopwords
    stemmed_sentence = [stemmer.stem(word)
              for word in tokenized_sentence] ## get word stems
    decoded_sentence = [unidecode.unidecode(w) for w in stemmed_sentence] ## remove accents

    cleaned_sentence = ' '.join(decoded_sentence) ## join back into a string

    return cleaned_sentence

In [7]:
df["full_review_cleaned"] = df["full_review"].apply(cleaning)
df["full_review_cleaned"]

0                               receb bem ant praz estipul
1        parabem loj lannist ador compr internet segur ...
2        recom aparelh efici sit marc aparelh impress d...
3                              pouc travandopel val ta boa
4           sup recom vend confi produt ok entreg ant praz
                               ...                        
36143                      produt receb acord compr realiz
36144    entreg dentr praz produt cheg cond perfeit sat...
36145    produt envi nf exist vend nf cert fic aguard e...
36146          excel mochil entreg sup rapid sup recom loj
36147          produt cheg ja devolv poi defeit segur carg
Name: full_review_cleaned, Length: 36148, dtype: object

## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

What is the proportion of reviews with a rating between 1 and 3 ? 

In [8]:
round(df["review_score"].value_counts(normalize = True),2)

5    0.57
4    0.16
1    0.13
3    0.09
2    0.05
Name: review_score, dtype: float64

Let's focus on these reviews...

In [9]:
threshold = 3
bad = df["review_score"] <= threshold
df = df[bad]
df.shape

(9658, 7)

In [10]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned
14,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,receb soment control mide split estil falt con...
23,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,compr dua unidad receb agor fac
24,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produt bom porem vei mim condiz fot anunci
25,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab
28,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz


### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this exercise.

In [11]:
vectorizer = TfidfVectorizer(max_df = 0.75, max_features = 5000, ngram_range=(1,2))
vectorized_reviews = pd.DataFrame(vectorizer.fit_transform(df["full_review_cleaned"]).toarray(),
                                 columns = vectorizer.get_feature_names_out())

print(f" vectorized_reviews.shape = {vectorized_reviews.shape}")

 vectorized_reviews.shape = (9658, 5000)


### (3.3) LDA

Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [12]:
n_components = 3

lda = LatentDirichletAllocation(n_components = n_components)
lda.fit(vectorized_reviews)

LatentDirichletAllocation(n_components=3)

#### Document Mixture (of Topics)

In [13]:
document_mixture = lda.transform(vectorized_reviews)
document_mixture.shape

(9658, 3)

In [14]:
round(pd.DataFrame(document_mixture,
                   columns = [f"Topic {i+1}" for i in range(n_components)])
      ,2)

,Topic 1,Topic 2,Topic 3
0,0.08,0.08,0.84
1,0.08,0.08,0.84
2,0.35,0.57,0.08
3,0.58,0.32,0.10
4,0.74,0.13,0.13
...,...,...,...
9653,0.08,0.11,0.81
9654,0.10,0.79,0.10
9655,0.07,0.13,0.80
9656,0.13,0.09,0.78


👉 Let's report the most important topic for each review

In [15]:
df["most_important_topic"] = np.argmax(document_mixture, axis = 1)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic
14,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,receb soment control mide split estil falt con...,2
23,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,compr dua unidad receb agor fac,2
24,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produt bom porem vei mim condiz fot anunci,1
25,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab,0
28,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz,0


#### Topic Mixture (of Words)

In [16]:
topic_mixture = pd.DataFrame(lda.components_,
                             columns = vectorizer.get_feature_names_out())
topic_mixture.shape

(3, 5000)

In [17]:
topic_mixture

abaix     abert   aborrec        abr  abr caix  abr cham  abr embal  \
0  2.282385  9.119152  2.467151   5.046816  1.008732  1.047649   1.786204   
1  2.282967  4.526988  0.378691   7.547169  0.899973  0.342657   2.099579   
2  0.416498  0.366747  0.539364  11.879877  1.276940  1.163182   0.532765   

   abr fech  abr reclam     abril    absurd  absurd entreg      abus  \
0  0.340461    0.340491  1.521299  7.575293       1.138091  0.409935   
1  1.762772    0.337296  0.462348  0.375096       0.334826  0.562470   
2  0.341954    5.972587  0.571924  5.442616       1.384346  0.949254   

        aca       acab  acab compr  acab deix  acab fic  acab produt  \
0  0.334869   7.025668    1.212645   0.341858  1.181390     0.335147   
1  0.340344  27.234719    0.552055   2.712829  1.103380     1.918044   
2  1.325550   0.554248    0.635152   0.334047  0.344173     0.335056   

   acab receb  acab recom  acab ruim     aceit     acend     acert     acess  \
0    1.030911    1.537472   0.334895  1.193190  0.345129  0.389246  1.604554   
1    0.365417    0.492693   3.966844  1.304884  2.671278  0.346668  1.311806   
2    1.684104    0.337174   0.334153  5.334811  0.335188  1.934557  0.782886   

   acessori        ach  ach absurd   ach bem   ach car  ach cord  ach demor  \
0  0.630045  38.250626    1.443047  0.763653  1.597081  0.334540   5.367902   
1  0.440119  34.517567    0.335492  1.775237  0.377958  1.128126   0.334632   
2  3.892083   0.653503    0.334657  0.335434  0.337900  0.333720   0.334908   

    ach dev  ach entreg  ach estranh  ach fornec  ach fret   ach loj  \
0  2.669653    2.407444     0.345308    1.239944  2.657979  0.641057   
1  2.684609    0.346339     1.799309    0.333846  0.340423  0.345547   
2  0.355892    0.343499     0.362946    0.335951  0.337776  1.892194   

   ach mant  ach mater  ach melhor  ach pequen  ach pess  ach pouc  ach praz  \
0  2.113075   0.339415    0.956555    4.759069  0.849584  4.645426  3.245548   
1  0.346263   3.115658    1.472821    0.366477  1.389324  0.361757  0.337788   
2  0.334224   0.336504    0.337635    0.343038  0.476339  0.334333  0.336042   

   ach prec  ach produt  ach qual   ach vei      acid      acim     acion  \
0  0.336552    6.270552  0.335507  2.201630  0.343172  0.347813  0.339183   
1  2.609398    3.745874  2.256713  0.354225  1.824487  0.639477  1.098637   
2  0.334305    0.339414  0.334120  0.350193  0.334091  1.941302  2.132744   

        aco  acompanh  acompanh ped  acompanh produt  acondicion    acontec  \
0  0.336135  1.471020      2.155047         0.729843    0.477846   5.787514   
1  2.317931  3.696653      0.336989         1.874574    2.644703   0.550407   
2  0.377569  4.658484      0.540554         0.376639    0.349486  13.680628   

   acontec compr  acontec produt     acord  acord descr   acostum  \
0       0.795022        2.883915  3.433917     0.336280  2.394399   
1       0.353825        0.342688  6.024026     1.229876  0.356982   
2       1.050622        0.381368  2.360056     0.696297  0.343957   

   acostum compr   acredit     acril  acril fic      acus     adapt     adequ  \
0       1.616750  4.288104  0.345591   0.334320  0.337445  3.025466  1.159753   
1       0.335361  5.170896  4.905545   1.905038  0.579585  2.428383  3.814212   
2       0.338826  3.255884  0.334785   0.334634  2.780110  0.445793  1.263260   

       ader      ades  ades pared  ades pra    adiant      ador    adquir  \
0  0.338725  4.637018    1.892486  0.333819  0.485952  2.365771  1.354281   
1  2.217397  5.899385    0.336440  1.109393  1.794846  8.630752  7.522145   
2  0.335223  0.446438    0.391219  0.333702  0.938538  0.340131  7.092975   

   adquir consegu  adquir produt     adult      afet  afet ades       aff  \
0        0.334327       0.341833  0.339284  0.723361   0.333819  0.993966   
1        1.671445       1.092608  0.424771  1.810323   1.109393  0.745199   
2        0.345231       2.588273  1.675870  0.498665   0.333702  0.339098   

     afinal     afirm    

#### Topics

We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [18]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [19]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))


🕵🏻‍♂️ Print the topics with their topwords:

In [20]:
print_topics(vectorizer, lda, topwords = 5)

--------------------
Topic 0:
[('produt', 166.02), ('entreg', 157.49), ('praz', 129.33), ('cheg', 124.88), ('recom', 76.49)]
--------------------
Topic 1:
[('bom', 211.17), ('produt', 167.11), ('vei', 153.54), ('qual', 104.77), ('gost', 75.92)]
--------------------
Topic 2:
[('receb', 327.43), ('compr', 206.47), ('produt', 188.66), ('entreg', 166.85), ('receb produt', 141.14)]


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _produto = product_
- _recomendo = recommend (não recomendo == not recommend)_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _errado = wrong_
- _gostaria = I would like to..._
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [21]:
topic_word_mixture = [topic_word(vectorizer, lda, topic, topwords = 5, with_weights = False)
                      for topic in lda.components_]
topic_word_mixture

[['produt', 'entreg', 'praz', 'cheg', 'recom'],
 ['bom', 'produt', 'vei', 'qual', 'gost'],
 ['receb', 'compr', 'produt', 'entreg', 'receb produt']]

In [22]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

In [23]:
df[["review_id",
        "review_score",
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head()

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
14,e233e51d11511bf30e568c76360ace52,1,eletronicos,receb soment control mide split estil falt con...,2,"[receb, compr, produt, entreg, receb produt]"
23,8b230a1373c6dc4bd867099fda1d7039,3,ferramentas_jardim,compr dua unidad receb agor fac,2,"[receb, compr, produt, entreg, receb produt]"
24,cb2fc3e5711b5ae85e0491ee18af63ed,3,beleza_saude,produt bom porem vei mim condiz fot anunci,1,"[bom, produt, vei, qual, gost]"
25,60c714ed14cef913944a3147094a4742,1,moveis_decoracao,produt inferi mal acab,0,"[produt, entreg, praz, cheg, recom]"
28,0bd4dcc4f6c4621baf37f73495cad8c4,3,esporte_lazer,entreg praz,0,"[produt, entreg, praz, cheg, recom]"


## (3.4) Pipeline Tf-Idf and LDA

In [24]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [25]:
%%time
# Parameters
max_df = 0.75
max_features = 5000
ngram_range = (1,2)

# Pipeline Vectorizer + LDA
pipeline = make_pipeline(
    TfidfVectorizer(max_df = max_df,
                    max_features = max_features,
                    ngram_range = ngram_range),
    LatentDirichletAllocation(n_components = n_components)
)

# Fit the pipeline on the cleaned texts
pipeline.fit(df["full_review_cleaned"])

CPU times: user 13.5 s, sys: 1.44 ms, total: 13.5 s
Wall time: 13.5 s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.75, max_features=5000,
                                 ngram_range=(1, 2))),
                ('latentdirichletallocation',
                 LatentDirichletAllocation(n_components=3))])

💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [26]:
pipeline._final_estimator

LatentDirichletAllocation(n_components=3)

In [27]:
pipeline._final_estimator.components_

array([[ 0.41588826,  0.36565385,  0.92323853, ...,  2.34639063,
         4.8674546 ,  0.36314216],
       [ 0.34958248,  0.38506238,  1.00655959, ...,  0.37682395,
         4.00528896,  1.03809588],
       [ 4.21637963, 13.26217106,  1.45540733, ...,  0.76588504,
         0.37761865,  5.55718738]])

**Document Mixture** with the Pipeline:

In [28]:
# Transform the original cleaned texts with the pipeline
# Indeed, there is no need to get the vectorized texts first since it's done through the Pipeline
document_mixture = pipeline.transform(df["full_review_cleaned"])
document_mixture.shape

(9658, 3)

**Topic Mixture** with the Pipeline:

In [29]:
topic_mixture = pd.DataFrame(pipeline._final_estimator.components_)
topic_mixture.shape

(3, 5000)

In [30]:
topic_mixture

0          1         2          3         4         5         6     \
0  0.415888   0.365654  0.923239  12.068105  1.303908  0.524313  0.423311   
1  0.349582   0.385062  1.006560   0.459775  0.349081  1.206232  0.362205   
2  4.216380  13.262171  1.455407  11.945983  1.532656  0.822944  3.633032   

       7         8         9         10        11        12        13    \
0  0.338570  5.953605  1.863984  5.742360  0.350632  0.952423  1.322340   
1  0.342777  0.357644  0.335602  5.004778  2.162069  0.536733  0.336784   
2  1.763839  0.339126  0.355985  2.645867  0.344561  0.432502  0.341639   

        14        15        16        17        18        19        20    \
0   0.382740  0.343775  0.334016  0.339442  0.335299  0.583173  0.334304   
1   0.346336  0.334935  0.334815  0.350236  0.334684  0.432449  0.334845   
2  34.085559  1.721141  2.719903  1.939264  1.918264  2.064811  1.698189   

       21        22        23        24        25        26         27    \
0  0.334714  4.734033  0.339898  0.570860  1.891131  3.825203   1.101586   
1  0.334925  1.801434  0.341936  1.710891  0.385715  0.432433   9.558063   
2  3.966252  1.297418  2.669760  0.388719  1.422399  0.704611  62.762046   

       28        29        30        31        32        33        34    \
0  0.646917  0.334634  0.353851  0.333760  0.337969  0.388043  0.437597   
1  0.405408  0.334950  0.335721  0.334107  5.334514  1.806419  2.260866   
2  1.060871  2.204741  1.623368  1.128519  0.364959  3.515692  0.398820   

       35        36        37        38        39        40        41    \
0  0.348717  1.240490  0.345593  0.340833  2.093332  0.335337  0.344524   
1  0.338475  0.334938  0.337756  0.673674  0.341060  0.354819  0.363442   
2  1.820371  0.334312  2.652830  1.864292  0.359171  3.101422  2.059046   

       42        43        44        45        46        47        48    \
0  0.334285  0.487544  0.336504  0.344915  0.336254  0.340247  0.334674   
1  0.340505  0.348096  0.344103  0.339956  1.312699  1.815491  2.174347   
2  4.793795  1.879608  4.660909  3.234507  1.631302  8.200102  0.417319   

       49        50        51        52        53        54        55    \
0  0.356331  0.335293  1.830513  1.639863  1.027205  2.910818  0.971867   
1  0.352246  0.335309  0.723094  0.373257  0.337301  1.471071  1.718221   
2  2.197471  1.831147  0.374984  1.557444  1.667130  5.444268  0.342501   

       56        57         58        59        60        61        62    \
0  0.358572  0.337899  18.021433  1.483208  0.377880  1.688863  0.360263   
1  0.335484  0.336009   1.357794  0.357470  2.893964  0.429158  0.342165   
2  2.287000  2.798126   0.639322  0.358791  0.336126  9.699978  1.560024   

       63        64         65        66        67        68        69    \
0  0.338755  0.345814   1.675686  0.338615  0.334000  1.707076  1.279068   
1  1.487200  1.296443   0.378510  0.341717  0.334438  1.640660  1.261200   
2  1.269383  0.648680  10.660688  4.905589  1.905554  0.349404  3.359374   

       70        71        72        73        74        75         76    \
0  0.376953  0.334976  3.329400  1.939480  0.333819  1.980571   0.363662   
1  0.569660  0.390057  0.388635  0.336662  0.334053  0.855952   0.342779   
2  5.290611  2.166311  7.264807  0.344002  1.109042  0.382814  10.630212   

        77        78        79        80        81        82        83    \
0  11.378895  1.681395  2.902564  1.728327  0.335290  0.333819  0.640382   
1   1.697030  0.335524  0.595817  0.351638  0.335514  0.334053  0.372676   
2   2.893475  0.334082  0.524334  0.359960  2.361545  1.109042  1.065205   

       84        85        86        87        88        89         90    \
0  1.521450  0.876191  8.475210  6.083865  0.339072  0.342884  47.879250   
1  0.346255  0.366525  0.338089  0.338899  5.240723  0.546510   9.666167   
2  0.607855  1.294973  0.392390  0.369611  0.380392  1.733418   0.358139   

       91        92        93        94        95        9

## (4) Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [31]:
# Product categories by performance - look at the count, mean, median and std
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Removing products which were sold less than a certain times for the analysis
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Sorting the product categories by performance
product_categories = product_categories.sort_values(by = [('review_score', 'mean'),
                                                          ('review_score', 'std')],
                                                    ascending = [False, True])
product_categories

review_score                           
                                         count      mean median       std
product_category_name                                                    
consoles_games                              92  1.956522    2.0  0.924787
fashion_bolsas_e_acessorios                155  1.948387    2.0  0.917319
malas_acessorios                            66  1.924242    2.0  0.949666
papelaria                                  155  1.922581    2.0  0.922556
casa_construcao                             66  1.909091    2.0  0.836242
utilidades_domesticas                      586  1.889078    2.0  0.888473
cool_stuff                                 298  1.885906    2.0  0.906618
relogios_presentes                         589  1.881154    2.0  0.905088
pet_shop                                   147  1.857143    2.0  0.883641
moveis_escritorio                          261  1.854406    1.0  0.920843
cama_mesa_banho                           1221  1.850942    2.0  0.890235
esporte_lazer                              626  1.846645    2.0  0.899136
casa_conforto                               57  1.842105    2.0  0.902169
beleza_saude                               716  1.840782    1.0  0.908275
ferramentas_jardim                         318  1.817610    1.0  0.904635
telefonia                                  488  1.815574    2.0  0.875636
moveis_decoracao                           743  1.802153    1.0  0.888620
brinquedos                                 298  1.788591    1.0  0.890808
bebes                                      244  1.786885    1.0  0.877046
eletronicos                                248  1.778226    1.0  0.883502
construcao_ferramentas_construcao           71  1.774648    1.0  0.881511
automotivo                                 370  1.770270    1.0  0.870275
perfumaria                                 235  1.723404    1.0  0.879543
informatica_acessorios                     797  1.708908    1.0  0.867089
eletrodomesticos                            84  1.630952    1.0  0.875087
eletroportateis                             54  1.518519    1.0  0.770708

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [32]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                              count      mean median       std
product_category_name                                         
informatica_acessorios          797  1.708908    1.0  0.867089
automotivo                      370  1.770270    1.0  0.870275
perfumaria                      235  1.723404    1.0  0.879543
eletrodomesticos                 84  1.630952    1.0  0.875087
eletroportateis                  54  1.518519    1.0  0.770708

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [33]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id",
                        "review_score",
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
34,ff722b4c68783a4459a3adb9bb4e1d0d,3,informatica_acessorios,produt cheg pc consegu reconhec port usb,1,"[bom, produt, vei, qual, gost]"
57,5f938e5f5f2e9a75710b54feeb9ea610,1,eletrodomesticos,medi pec serv,1,"[bom, produt, vei, qual, gost]"
71,6b341682ab39af9fa00d72d7388c903b,3,automotivo,not compr produt corret cap intern outr,2,"[receb, compr, produt, entreg, receb produt]"
88,b736ff4204044e49e39062584d06fa74,1,eletroportateis,loj anunc produt entreg outr,2,"[receb, compr, produt, entreg, receb produt]"
100,6f885e2fd69412264d9e74e805175d53,1,informatica_acessorios,receb bem rapid porem vei outr produt ped mode...,1,"[bom, produt, vei, qual, gost]"
...,...,...,...,...,...,...
36049,e9f0bde9a98ff79305964e9033ed8cd2,1,informatica_acessorios,produt cheg falt pess cond,0,"[produt, entreg, praz, cheg, recom]"
36054,44380245bf1a49875542ea0e7f216986,1,informatica_acessorios,receb produt aind assim receb produt avali posi,2,"[receb, compr, produt, entreg, receb produt]"
36095,970ae7c9df61910a0a30c38670db7ae5,3,informatica_acessorios,probl past multius pra notebook ped cor vermel...,0,"[produt, entreg, praz, cheg, recom]"
36100,504403288b387de1c95a193909ba5dfd,3,perfumaria,cheg bem rapid produt razoa,0,"[produt, entreg, praz, cheg, recom]"


### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [34]:
worst_products_reviews["most_important_topic"].value_counts()

2    650
0    446
1    444
Name: most_important_topic, dtype: int64

In [35]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

[2, 0, 1]

In [36]:
[topic_word_mixture[i] for i in bad_frequency]

[['receb', 'compr', 'produt', 'entreg', 'receb produt'],
 ['produt', 'entreg', 'praz', 'cheg', 'recom'],
 ['bom', 'produt', 'vei', 'qual', 'gost']]

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this with Decision Making
